<h1 style="font-size:3rem;color:darkred;">Predicting House Prices with Linear Regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

### Data Preparation and Exploration

In [ ]:
# Features: [size (1000 sqft), number of bedrooms, age of house (years)]
X = np.array([[1.0, 3, 10], [1.5, 3, 15], [2.0, 4, 5], [2.5, 4, 12], [3.0, 5, 8], [3.2, 5, 20]])
y = np.array([300, 350, 480, 430, 630, 730]) # Prices in $1000s

print("X.shape:", X.shape)
print("y.shape:", y.shape)

### Implement the Cost Function

In [ ]:
def compute_cost(X, y, w, b):
    m = X.shape[0] 
    total_cost = 0
    for i in range(m):
        f_wb = np.dot(X[i], w) + b
        cost = (f_wb - y[i]) ** 2
        total_cost += cost
    total_cost = total_cost / (2 * m)
    return total_cost

# Test your function
initial_w = np.zeros(X.shape[1])
initial_b = 0
cost = compute_cost(X, y, initial_w, initial_b)
print(f"Initial cost: {cost}")

### Implement Gradient Descent

In [ ]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iters):
    m, n = X.shape
    w = w_in.copy()
    b = b_in
    for i in range(num_iters):
        dj_dw = np.zeros(n)
        dj_db = 0
        for j in range(m):
            err = (np.dot(X[j], w) + b) - y[j]
            for k in range(n):
                dj_dw[k] += err * X[j][k]
            dj_db += err
        w = w - alpha * dj_dw / m
        b = b - alpha * dj_db / m
    return w, b

# Run gradient descent:
iterations = 1000
alpha = 0.01
w, b = gradient_descent(X, y, initial_w, initial_b, alpha, iterations)
print(f"Final w: {w}, Final b: {b}")

### Feature Scaling

In [ ]:
# Implement Z-score normalization
def normalize_features(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

X_normalized = normalize_features(X)

# Re-run gradient descent:
#iterations = 1000
#alpha = 0.01
w_norm, b_norm = gradient_descent(X_normalized, y, initial_w, initial_b, alpha, iterations)
print(f"Normalized: Final w: {w_norm}, Final b: {b_norm}")

###  Make Predictions

In [ ]:
# Write prediction function
def predict(X, w, b):
    return np.dot(X, w) + b

# Make predicitons for a new house
new_house = np.array([2.8, 4, 18])  # 2800 sqft, 4 bedrooms, 18 years old
new_house_normalized = (new_house - np.mean(X, axis=0)) / np.std(X, axis=0)
predicted_price = predict(new_house_normalized, w_norm, b_norm)
print(f"Predicted Price for the new house: ${predicted_price*1000:.2f}")

### Vectorization

In [ ]:
# Reimplement the cost function and gradient descent using vectorization:
def compute_cost_vectorized(X, y, w, b):
    m = X.shape[0]
    f_wb = np.dot(X, w) + b
    total_cost = np.sum((f_wb - y) ** 2) / (2 * m)
    return total_cost

def gradient_descent_vectorized(X, y, w_in, b_in, alpha, num_iters):
    m, n = X.shape
    w = w_in.copy()
    b = b_in
    for i in range(num_iters):
        f_wb = np.dot(X, w) + b
        dj_dw = np.dot(X.T, (f_wb - y)) / m
        dj_db = np.sum(f_wb - y) / m
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
    return w, b

# Compare execution time
start_time = time.time()
w_vect, b_vect = gradient_descent_vectorized(X_normalized, y, initial_w, initial_b, alpha, iterations)
vect_time = time.time() - start_time

start_time = time.time()
w_non_vect, b_non_vect = gradient_descent(X_normalized, y, initial_w, initial_b, alpha, iterations)
loop_time = time.time() - start_time

print(f"Vectorized time: {vect_time:.6f} seconds")
print(f"Non-vectorized time: {loop_time:.6f} seconds")

### Analysis and Conclusion

#### Cost Vs Iterations plot for Vectorized and non-vectorized implementation

In [ ]:
def gradient_descent_vectorized_cost(X, y, w_in, b_in, alpha, num_iters):
    m, n = X.shape
    w = w_in.copy()
    b = b_in
    cost_history = []
    for i in range(num_iters):
        f_wb = np.dot(X, w) + b
        dj_dw = np.dot(X.T, (f_wb - y)) / m
        dj_db = np.sum(f_wb - y) / m
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        cost_history.append(compute_cost(X, y, w, b))
    return cost_history

In [ ]:
def gradient_descent_cost(X, y, w_in, b_in, alpha, num_iters):
    m, n = X.shape
    w = w_in.copy()
    b = b_in
    cost_hist = []
    for i in range(num_iters):
        dj_dw = np.zeros(n)
        dj_db = 0
        for j in range(m):
            err = (np.dot(X[j], w) + b) - y[j]
            for k in range(n):
                dj_dw[k] += err * X[j][k]
            dj_db += err
        w = w - alpha * dj_dw / m
        b = b - alpha * dj_db / m
        cost_hist.append(compute_cost(X, y, w, b))
    return cost_hist

In [ ]:
cost_vect = gradient_descent_vectorized_cost(X_normalized, y, initial_w, initial_b, alpha, iterations)
cost_non_vect = gradient_descent_cost(X_normalized, y, initial_w, initial_b, alpha, iterations)

plt.figure(figsize = (14, 6))
# Vectorized Plot
plt.subplot(1,2,1)
plt.plot(range(iterations), cost_vect, label = 'Vectorized', linestyle = '--', color = 'darkblue', alpha=0.9)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.title('Cost Vs Iterations (Vectorized Implementation)')
plt.legend()
plt.grid(True)

# Non_Vectorized Plot
plt.subplot(1,2,2)
plt.plot(range(iterations), cost_non_vect, label = 'Non-Vectorized', linestyle = '-', color = 'red', alpha=0.4)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.title('Cost Vs Iterations (Non-Vectorized Implementation)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


#### Impact of Feature Scaling on Convergence of Gradient Descent

Feature scaling adjusts the features so they are on a similar scale.  In this code, we used the Z-score normalization technique.  This transformation centers the features around 0, with a standard deviation of 1.  Without feature scaling, the gradient descent algorithm convergence may take longer due to the imbalanced feature magnitude. Features like "size(1.0 - 3.2)" are much smaller than "age of house(5 - 20)" and would result in weights that are incomparable in magnitude.  Hence, it can be implied that feature scaling leads to faster convergence, improved numerical stability and makes scaled weights easier to interpret. 

#### Comparison of Efficiency of Vectorized and Non-Vectorized Implementations

The implementation was on a small dataset (6 X 3), and the difference in execution time between the vectorized and non-vectorized implementations was negligible.  The result of the execution timing code above revealed the vectorized code execution at 0.012000 and the non-vectorized code at 0.015009 seconds. For a large dataset, the vectorized implementation will complete faster than the non-vectorized and this can be noted from the Lab(2), where we tested vectorization against non-vectorization with a large dataset. In comparison to the non-vectorized implementation, the vectorized implementation is more efficient for some reasons below;

- It eliminates the reliance on python loop overheads:  Non-vectorized implementation relies heavily on python loops, which introduce this overhead for each iteration.
- Quicker processing:  In vectorized implementation, operations are performed on entire matrices or vectors at once and not by looping, thereby reducing time and the overhead of loop iterations.

#### Some Limitations on the model

1. Assumption of linearity:  The model assumes a linear relationship between the features and target variable, and this might not necessarily be the case for all dataset.  If non-linear relationship is established, then the models accuracy will suffer.
2. Limited Dataset:  The size of the dataset is too small and would limit the performance of the model. This limited size of the data could also lead to overfitting or underfitting in another instance.  The larger the dataset, the better the model performance will be.
3. Limited features from the dataset.                         

#### Suggested Improvements

Based on the above limitations some suggestions on model improvement include, but not limited to the following;
1. Increase the dataset:  A larger dataset will improve the models ability to generalize and thereby reduce risk of overfitting or underfitting.  An increased dataset will improve the optimization process in the gradient descent and give the model more data to learn fro, thereby improving the overall accuracy of the model.
2. Incorporate Additional Features:  The model currently uses only size, number of bedrooms, and age.  Additional features can significantly improve model accuracy.
3. Feature Engineering:  We could consider introducing feature engineering to capture the more complex relationships between the existing features or even creating new ones from the original dataset.